# REDES NEURAIS ARTIFICIAIS
## Evaluate Improve e Tunning

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.6.0'

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [5]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [9]:
tf.random.set_seed(0) #Define que os casos de teste e os casos para praticar serão sempre iguais

In [10]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [11]:
ann.add(tf.keras.layers.Dense(units=6, activation='tanh'))

### Adding the second hidden layer

In [12]:
ann.add(tf.keras.layers.Dense(units=6, activation='tanh'))

### Adding the output layer

In [13]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [14]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [15]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 867us/step - loss: 0.6061 - accuracy: 0.6874
Epoch 2/100
250/250 [==============================] - 0s 791us/step - loss: 0.4607 - accuracy: 0.8046
Epoch 3/100
250/250 [==============================] - 0s 755us/step - loss: 0.4288 - accuracy: 0.8116
Epoch 4/100
250/250 [==============================] - 0s 719us/step - loss: 0.4199 - accuracy: 0.8165
Epoch 5/100
250/250 [==============================] - 0s 799us/step - loss: 0.4136 - accuracy: 0.8223
Epoch 6/100
250/250 [==============================] - 0s 759us/step - loss: 0.4076 - accuracy: 0.8250
Epoch 7/100
250/250 [==============================] - 0s 731us/step - loss: 0.4009 - accuracy: 0.8296
Epoch 8/100
250/250 [==============================] - 0s 819us/step - loss: 0.3939 - accuracy: 0.8319
Epoch 9/100
250/250 [==============================] - 0s 775us/step - loss: 0.3869 - accuracy: 0.8355
Epoch 10/100
250/250 [==============================] - 0s 972us/step - l

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [16]:
y_pred = ann.predict(X_test)
y_pred

array([[0.28365082],
       [0.16923717],
       [0.10380057],
       ...,
       [0.11263412],
       [0.174887  ],
       [0.24651918]], dtype=float32)

In [17]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [18]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1527   68]
 [ 202  203]]


In [19]:
#cm[0, 0]: Previsão correta op1
#cm[0, 1]: Previsão incorreta op1
#cm[1, 0]: Previsão incorreta op2
#cm[1, 1]: Previsão correta op2

# Funções de ativação experimentadas: relu, softmax, tanh

#Acurácia das combinações
# relu + relu: 0,86
# relu + softmax: 0,8585
# relu + tanh: 0,8605

# softmax + relu: 0,8635
# softmax + softmax: 0,8575
# softmax + tanh: 0,859

# tanh + relu: 0,8605
# tanh + softmax: 0,8605
# tanh + tanh: 0,865

from sklearn.metrics import (accuracy_score)
accuracy_score(y_test, y_pred)

0.865

---
# Avaliando o Modelo

In [20]:
# Evaluating the ANN
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [21]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 12))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [22]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [28]:
print(mean,variance)

0.8487500071525573 0.009185589131014232


# Melhorando o modelo

In [29]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed
from tensorflow.keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [30]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [31]:
print(mean,variance)

0.832749992609024 0.014315645780056934


# Tuning the ANN

In [32]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

# O grid search não será executado por demorar demais em comparação com o Halving Search, mas mantive esse pedaço aqui para saber como utilizar caso necessário

# Halving Grid Search

In [33]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

# Adicionando mais parâmetros
parameters = {
    'batch_size': [10, 25, 32, 50],
    'epochs': [50, 100, 150],
    'optimizer': ['adam', 'rmsprop', 'adamax', 'adagrad']
}

halving_cv = HalvingGridSearchCV(estimator = classifier,
                                  param_grid = parameters,
                                  scoring="accuracy", 
                                  n_jobs=-1, 
                                  min_resources="exhaust", 
                                  factor=3,
                                  cv = 10)

In [36]:
%%time
halving_cv = halving_cv.fit(X_train, y_train)

Epoch 1/100
800/800 [==============================] - 1s 722us/step - loss: 0.4819 - accuracy: 0.8086
Epoch 2/100
800/800 [==============================] - 1s 732us/step - loss: 0.4374 - accuracy: 0.8080
Epoch 3/100
800/800 [==============================] - 1s 752us/step - loss: 0.4353 - accuracy: 0.8114
Epoch 4/100
800/800 [==============================] - 1s 677us/step - loss: 0.4339 - accuracy: 0.8112
Epoch 5/100
800/800 [==============================] - 1s 710us/step - loss: 0.4339 - accuracy: 0.8114
Epoch 6/100
800/800 [==============================] - 1s 708us/step - loss: 0.4305 - accuracy: 0.8133
Epoch 7/100
800/800 [==============================] - 1s 760us/step - loss: 0.4297 - accuracy: 0.8117
Epoch 8/100
800/800 [==============================] - 1s 742us/step - loss: 0.4204 - accuracy: 0.8190
Epoch 9/100
800/800 [==============================] - 1s 723us/step - loss: 0.4100 - accuracy: 0.8255
Epoch 10/100
800/800 [==============================] - 1s 711us/step - l

In [37]:
print('Best params:\n',halving_cv.best_params_,'\nBest accuracy:\n',halving_cv.best_score_)

Best params:
 {'batch_size': 10, 'epochs': 100, 'optimizer': 'adam'} 
Best accuracy:
 0.8569461827284105


In [ ]:
#Obs: com a adição de parâmetros, a acurácia aumentou, apesar dos melhores parâmetros não terem alterado

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)

Modificado por
> Álvaro Leles Guimarães